# MIIC

Prepare files to send on the web platform

# 0. Loading the libraries

In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

2022-06-17 14:20:28.003292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/R/lib/R/lib:/opt/R/lib/R/lib:
2022-06-17 14:20:28.003382: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# Set up constants
save_folder = "figures/"
objects_folder = "saved_objects/"
sc.settings.figdir = './'+save_folder

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3

# 1. MIIC

In [33]:
with open('saved_objects/adata_annotated.pkl', 'rb') as inp:
    adata = pickle.load(inp)

adata.X = adata.raw.X

In [ ]:
def miic(lgenes, data):

    gene_data = pd.DataFrame(columns = lgenes)
    for gene in lgenes:
        gene_data[gene] = [e[0] for e in data.X[:, data.var.gene_name == gene].tolist()]
    miic_data = pd.concat([ gene_data.reset_index(drop=True), pd.DataFrame(data.obs[['Il10_positive', 'Il10_high']]).reset_index(drop=True) ], axis=1)

    miic_data.to_csv("miic/"+name+"_data.csv", header = True, index = False)
    miic_data.to_excel("miic/"+name+"_data.xlsx", header = True, index = False)

    supp = pd.DataFrame(columns = ['var_names', 'var_type', 'levels_increasing_order', 'is_contextual', 'group'])
    supp['var_names'] = miic_data.columns
    supp['var_type'] = [1]*len(lgenes) + [0]*(len(miic_data.columns) - len(lgenes))
    supp['levels_increasing_order'] = ['']*len(lgenes) + ['Il10-,Il10+', 'Not,High']
    supp['is_contextual'] = [0]*len(lgenes) + [1]*(len(miic_data.columns) - len(lgenes))
    supp['group'] = 'all'

    supp.to_csv("miic/"+name+"_supp.tsv", sep = '\t', header = True, index = False)

## 1.1

In [26]:
data = adata[(adata.obs.sample2 == "N6-48h")].copy()
name = 'all-intergenes'
sig_genes_sheet = pd.read_excel("figures/"+"coex_inter_Il10"+".xlsx", sheet_name = "significant", header = 0).gene.tolist()[:90]
sig_genes = sig_genes_sheet+["Il10"]
L = list(set(sig_genes))
sig_genes = [x for x in L if x == x]